# Full Fine-Tuning: Qwen2.5-7B on AG News

This notebook performs **full fine-tuning** (updating all 7B parameters) of the Qwen2.5-7B-Instruct model on the AG News classification dataset.

## What is Full Fine-Tuning?

Unlike LoRA/QLoRA which only trains small adapter layers, **full fine-tuning** updates ALL model parameters:

| Aspect | Full Fine-Tuning | LoRA |
|--------|------------------|------|
| Parameters updated | 7,000,000,000 (100%) | ~70,000,000 (1%) |
| Memory required | ~60-70 GB | ~16-24 GB |
| Training time | 8-15 hours | 2-4 hours |
| Output size | ~14 GB | ~100-500 MB |
| Risk of overfitting | Higher | Lower |
| Potential improvement | Higher | Moderate |

## Environment

**This notebook is designed to run inside the NVIDIA PyTorch Docker container** (`nvcr.io/nvidia/pytorch:25.11-py3`) which provides:
- Native sm_120/121 CUDA kernels (optimized for Blackwell GB10)
- Transformer Engine 2.9+ with FP8 support
- Flash Attention 2 compiled for Blackwell
- Triton with optimized kernels

## Prerequisites

Before running this notebook:

1. **Start the training container**:
   ```bash
   cd ~/Projects/xiaohui-agentic-playground/6-open-source
   ./start_docker.sh start finetune
   ```

2. **Open Jupyter** at http://localhost:8888

3. **Training data prepared** at `datasets/train.jsonl` (120K samples)

## Expected Results

- **Base model accuracy**: 78.63%
- **Target accuracy**: 85-92% (with fine-tuning)
- **Training time**: ~7-9 hours for 1 epoch (with optimizations on DGX Spark)

## 1. Environment Setup & Pre-flight Checks

First, let's verify that:
- GPU is available and has sufficient memory
- Required libraries are installed
- Training data exists

In [ ]:
# Pre-flight checks
import torch
import os
from pathlib import Path

# =============================================================================
# OPTIMIZATION 1: Enable cuDNN benchmark mode for consistent input sizes
# =============================================================================
torch.backends.cudnn.benchmark = True
print("✓ cuDNN benchmark mode enabled (optimization for consistent input sizes)")

print()
print("=" * 60)
print("PRE-FLIGHT CHECKS")
print("=" * 60)

# Check Docker environment
print(f"\n[1] Docker Environment:")
if os.path.exists("/usr/local/cuda-13.0") or os.path.exists("/usr/local/cuda"):
    print("    ✓ Running in NVIDIA PyTorch container")
    # Check for Transformer Engine
    try:
        import transformer_engine as te
        print(f"    ✓ Transformer Engine: {te.__version__}")
    except ImportError:
        print("    ⚠ Transformer Engine not found (optional)")
    # Check for Triton
    try:
        import triton
        print(f"    ✓ Triton: {triton.__version__}")
    except ImportError:
        print("    ⚠ Triton not found")
else:
    print("    ⚠ Not in NVIDIA container - may have suboptimal performance")
    print("    ⚠ Start with: ./start_docker.sh start finetune")

# Check GPU
print(f"\n[2] GPU Availability:")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    compute_cap = torch.cuda.get_device_capability(0)
    print(f"    ✓ CUDA available: {gpu_name}")
    print(f"    ✓ Compute capability: sm_{compute_cap[0]}{compute_cap[1]}")
    print(f"    ✓ Total memory: {gpu_memory:.1f} GB")
    
    # Check current memory usage
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"    ✓ Currently allocated: {allocated:.1f} GB")
    print(f"    ✓ Currently reserved: {reserved:.1f} GB")
else:
    print("    ✗ CUDA NOT available - cannot proceed!")

# Check training data
print(f"\n[3] Training Data:")
train_file = Path("datasets/train.jsonl")
if train_file.exists():
    size_mb = train_file.stat().st_size / 1e6
    with open(train_file) as f:
        num_lines = sum(1 for _ in f)
    print(f"    ✓ Found: {train_file}")
    print(f"    ✓ Size: {size_mb:.1f} MB")
    print(f"    ✓ Examples: {num_lines:,}")
else:
    print(f"    ✗ Training data not found at {train_file}")

# Check output directory
print(f"\n[4] Output Directory:")
output_dir = Path("checkpoints/qwen7b-ag-news-full")
output_dir.mkdir(parents=True, exist_ok=True)
print(f"    ✓ Will save to: {output_dir}")

print("\n" + "=" * 60)
print("PRE-FLIGHT CHECKS COMPLETE")
print("=" * 60)

In [ ]:
# Library versions - Docker container already has these installed
import transformers
import trl
import accelerate

print("Library versions:")
print(f"  torch: {torch.__version__}")
print(f"  transformers: {transformers.__version__}")
print(f"  trl: {trl.__version__}")
print(f"  accelerate: {accelerate.__version__}")

# Check for Transformer Engine (optional, for FP8)
try:
    import transformer_engine as te
    print(f"  transformer_engine: {te.__version__}")
except ImportError:
    print("  transformer_engine: not installed")

# Check CUDA version
print(f"\nCUDA info:")
print(f"  CUDA version (torch): {torch.version.cuda}")
print(f"  cuDNN version: {torch.backends.cudnn.version()}")

## 2. Configuration

Define all hyperparameters and settings for training.

### Key Hyperparameters Explained:

| Parameter | Value | Explanation |
|-----------|-------|-------------|
| `learning_rate` | 2e-5 | Lower than LoRA (1e-4) because we're updating all params |
| `num_train_epochs` | 1 | Full fine-tuning often needs fewer epochs |
| `per_device_train_batch_size` | 8 | Larger batch = better GPU utilization |
| `gradient_accumulation_steps` | 4 | Effective batch size = 8 × 4 = 32 |
| `gradient_checkpointing` | True | Trades compute for memory (essential!) |
| `bf16` | True | BFloat16 precision for speed + stability |
| `packing` | True | Pack multiple short sequences into one batch (big speedup!) |

### Performance Optimizations Applied

1. **cuDNN benchmark mode** - Auto-tunes kernels for consistent 512-token inputs
2. **Sequence packing** - Avg sequence ~120 tokens, packing eliminates ~75% wasted compute
3. **Flash Attention 2** - Memory-efficient attention optimized for Blackwell
4. **Larger batch size** - Better GPU utilization, fewer sync points
5. **DataLoader workers** - Parallel CPU preprocessing

### Expected Training Time

On DGX Spark with NVIDIA Docker container + optimizations: **~7-9 hours** for 1 epoch on 120K examples.

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Model
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "checkpoints/qwen7b-ag-news-full"

# Data
TRAIN_FILE = "datasets/train.jsonl"
MAX_SEQ_LENGTH = 512  # Our data is short (~120 tokens avg)

# Training hyperparameters
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
BATCH_SIZE = 8  # Per device (OPTIMIZATION 4: increased from 4 for better GPU utilization)
GRADIENT_ACCUMULATION_STEPS = 4  # Effective batch = 32 (reduced from 8, fewer sync points)
WARMUP_RATIO = 0.03
WEIGHT_DECAY = 0.01
MAX_GRAD_NORM = 1.0

# Checkpointing
SAVE_STEPS = 500
LOGGING_STEPS = 50
SAVE_TOTAL_LIMIT = 2  # Keep only 2 checkpoints to save disk space

# Memory optimization
USE_GRADIENT_CHECKPOINTING = True
USE_BF16 = True

# Performance optimizations
ENABLE_TORCH_COMPILE = False  # Set True for potential 10-20% speedup (experimental)
USE_PACKING = True  # OPTIMIZATION 2: Pack multiple short sequences into one batch
DATALOADER_NUM_WORKERS = 4  # OPTIMIZATION 5: Parallel data loading

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} effective")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  Sequence packing: {USE_PACKING}")
print(f"  DataLoader workers: {DATALOADER_NUM_WORKERS}")
print(f"  torch.compile: {ENABLE_TORCH_COMPILE}")

## 3. Load and Prepare Dataset

Load the prepared training data and format it for the SFTTrainer.

In [ ]:
from datasets import load_dataset

print("Loading training dataset...")
dataset = load_dataset("json", data_files=TRAIN_FILE, split="train")

print(f"\nDataset loaded:")
print(f"  Examples: {len(dataset):,}")
print(f"  Features: {dataset.features}")

# Show a sample
print(f"\nSample example:")
sample = dataset[0]
for msg in sample["messages"]:
    role = msg["role"]
    content = msg["content"][:100] + "..." if len(msg["content"]) > 100 else msg["content"]
    print(f"  [{role}]: {content}")

## 4. Load Base Model

Load the Qwen2.5-7B-Instruct model in BF16 precision.

**Important**: We enable `gradient_checkpointing` to reduce memory usage. This trades compute for memory by recomputing activations during backward pass instead of storing them.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"  Set pad_token to eos_token: {tokenizer.pad_token}")

print(f"\nTokenizer loaded:")
print(f"  Vocab size: {tokenizer.vocab_size:,}")
print(f"  Model max length: {tokenizer.model_max_length:,}")

In [ ]:
print("Loading model (this may take a few minutes)...")
print(f"  Model: {MODEL_NAME}")
print(f"  Precision: BF16")
print(f"  Gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")

# Determine best attention implementation
# Docker container has Flash Attention 2 compiled for Blackwell
try:
    import flash_attn
    attn_impl = "flash_attention_2"
    print(f"  Attention: Flash Attention 2 (optimized for Blackwell)")
except ImportError:
    attn_impl = "sdpa"
    print(f"  Attention: SDPA (Flash Attention 2 not available)")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation=attn_impl,
)

# Enable gradient checkpointing to save memory
if USE_GRADIENT_CHECKPOINTING:
    model.gradient_checkpointing_enable()
    print("  ✓ Gradient checkpointing enabled")

# Optional: torch.compile for potential speedup
if ENABLE_TORCH_COMPILE:
    print("  Compiling model with torch.compile (this may take a few minutes)...")
    model = torch.compile(model)
    print("  ✓ Model compiled")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel loaded:")
print(f"  Total parameters: {total_params:,} ({total_params/1e9:.2f}B)")
print(f"  Trainable parameters: {trainable_params:,} ({trainable_params/1e9:.2f}B)")
print(f"  Trainable %: {100 * trainable_params / total_params:.2f}%")

# Check memory usage
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    print(f"\nGPU memory after model load: {allocated:.1f} GB")

## 5. Training Configuration Preview

Preview the training configuration. The actual configuration will be set via `SFTConfig` in the next section.

In [ ]:
# Calculate expected training steps for reference
# Note: We'll use SFTConfig in the next cell which combines TrainingArguments + SFT options
num_training_steps = (len(dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_EPOCHS
warmup_steps = int(0.03 * num_training_steps)  # 3% warmup

print("Training Configuration Preview:")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} effective")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Total training steps: ~{num_training_steps:,}")
print(f"  Checkpoint every: {SAVE_STEPS} steps")
print(f"\nOptimizations enabled:")
print(f"  1. cuDNN benchmark mode: ✓")
print(f"  2. Sequence packing: {USE_PACKING}")
print(f"  3. Flash Attention 2: ✓ (set in model loading)")
print(f"  4. Batch size 8 / Accum 4: ✓")
print(f"  5. DataLoader workers: {DATALOADER_NUM_WORKERS}")
print(f"\n  Expected training time: ~7-9 hours (with optimizations)")

## 6. Initialize SFTTrainer

We use TRL's `SFTTrainer` (Supervised Fine-Tuning Trainer) with `SFTConfig` which handles:
- Chat template formatting
- **Assistant-only loss**: Only compute loss on assistant responses (via `assistant_only_loss=True`)
- Efficient data collation
- All training hyperparameters in one config object

In [ ]:
from trl import SFTTrainer, SFTConfig

# In TRL 0.27+, use SFTConfig with assistant_only_loss instead of DataCollatorForCompletionOnlyLM
# This automatically masks the loss so we only train on assistant responses

print("Initializing SFTTrainer with performance optimizations...")

# Create SFTConfig with all training parameters
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    
    # Training duration
    num_train_epochs=NUM_EPOCHS,
    
    # Batch size
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    
    # Optimizer
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_steps=int(0.03 * (len(dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS))),  # 3% warmup
    max_grad_norm=MAX_GRAD_NORM,
    optim="adamw_torch",
    
    # Precision
    bf16=USE_BF16,
    
    # Checkpointing
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    
    # Logging
    logging_steps=LOGGING_STEPS,
    logging_first_step=True,
    report_to="none",
    
    # Memory optimization
    gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
    
    # Note: assistant_only_loss=True requires tokenizer chat template with {% generation %} keyword
    # Qwen2.5's template doesn't support this, so we train on full sequences
    # This is acceptable since our assistant responses are very short (just category JSON)
    assistant_only_loss=False,
    
    # Sequence length (max_length in TRL 0.27+)
    max_length=MAX_SEQ_LENGTH,
    
    # =============================================================================
    # PERFORMANCE OPTIMIZATIONS
    # =============================================================================
    # OPTIMIZATION 2: Sequence packing - pack multiple short sequences into one batch
    # Our avg sequence is ~120 tokens, max is 512. Packing eliminates ~75% wasted compute!
    packing=USE_PACKING,
    
    # OPTIMIZATION 5: Parallel data loading
    dataloader_num_workers=DATALOADER_NUM_WORKERS,
    
    # Other
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)

print("\n✓ SFTTrainer initialized with optimizations:")
print(f"  Training examples: {len(trainer.train_dataset):,}")
print(f"  Loss computed on: {'assistant only' if sft_config.assistant_only_loss else 'full sequence'}")
print(f"  Sequence packing: {sft_config.packing}")
print(f"  DataLoader workers: {sft_config.dataloader_num_workers}")

# Final memory check
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    print(f"  GPU memory before training: {allocated:.1f} GB")

## 7. Train!

Now we start the training process. With optimizations enabled, this should take **~7-9 hours** for 1 epoch on 120K examples on DGX Spark.

**Optimizations Applied:**
1. cuDNN benchmark mode (5-10% speedup)
2. Sequence packing (30-40% speedup - biggest impact!)
3. Flash Attention 2 (10-15% speedup)
4. Larger batch size with less gradient accumulation (5-8% speedup)
5. Parallel data loading (3-5% speedup)

**What to monitor:**
- `loss` should decrease over time
- GPU memory usage (watch for OOM errors)
- Training speed (it/s) - expect ~0.12-0.14 it/s with optimizations

**Checkpoints are saved automatically** every 500 steps, so if training is interrupted, you can resume from the last checkpoint.

In [ ]:
import time

print("=" * 60)
print("STARTING TRAINING (WITH OPTIMIZATIONS)")
print("=" * 60)
print(f"\nModel: {MODEL_NAME}")
print(f"Dataset: {len(dataset):,} examples")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch: {BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} effective")
print(f"Packing: {USE_PACKING}")
print(f"Expected time: ~7-9 hours (with optimizations)")
print("\nCheckpoints will be saved to:", OUTPUT_DIR)
print("\n" + "=" * 60)

start_time = time.time()

# Train!
trainer.train()

end_time = time.time()
training_time = (end_time - start_time) / 3600  # Convert to hours

print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print(f"\nTotal training time: {training_time:.2f} hours")

## 8. Save the Fine-Tuned Model

Save the complete fine-tuned model (all weights, ~14 GB).

In [ ]:
print("Saving fine-tuned model...")

# Save the model
final_model_path = f"{OUTPUT_DIR}/final"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"\n✓ Model saved to: {final_model_path}")

# Check size
import subprocess
result = subprocess.run(["du", "-sh", final_model_path], capture_output=True, text=True)
print(f"✓ Model size: {result.stdout.strip()}")

## 9. Quick Evaluation

Let's do a quick sanity check by classifying a few examples with the fine-tuned model.

In [12]:
# Quick evaluation on a few examples
print("=" * 60)
print("QUICK EVALUATION")
print("=" * 60)

# Test examples (one from each category)
test_examples = [
    {"text": "President Biden announces new climate policy at UN summit", "expected": "World"},
    {"text": "Lakers defeat Celtics 115-108 in overtime thriller", "expected": "Sports"},
    {"text": "Apple stock rises 5% after strong quarterly earnings report", "expected": "Business"},
    {"text": "Google releases new AI model that can generate realistic images", "expected": "Sci/Tech"},
]

# System prompt (same as training)
system_prompt = """You are a news article classifier. Your task is to categorize news articles into exactly one of four categories:

- World: News about politics, government, elections, diplomacy, conflicts, and public affairs (domestic or international)
- Sports: News about athletic events, games, players, teams, coaches, tournaments, and championships
- Business: News about companies, markets, finance, economy, trade, corporate activities, and business services
- Sci/Tech: News about technology products, software, hardware, scientific research, gadgets, and tech innovations

Rules:
- Focus on the PRIMARY topic of the article
- Ignore HTML artifacts (like #39; or &lt;b&gt;) - they are formatting errors
- If an article is truncated, classify based on the available content
- When a topic spans multiple categories, choose the one that best represents the main focus"""

model.eval()

for example in test_examples:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Classify the following news article:\n\n{example['text']}"}
    ]
    
    # Note: return_dict=False returns just the tensor, not a BatchEncoding object
    inputs = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt",
        add_generation_prompt=True,
        return_dict=False,
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=50,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # inputs is now a tensor, so .shape works correctly
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    print(f"\nArticle: {example['text'][:60]}...")
    print(f"Expected: {example['expected']}")
    print(f"Predicted: {response.strip()}")
    print("-" * 40)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


QUICK EVALUATION

Article: President Biden announces new climate policy at UN summit...
Expected: World
Predicted: {"category":"World"}
----------------------------------------

Article: Lakers defeat Celtics 115-108 in overtime thriller...
Expected: Sports
Predicted: {"category":"Sports"}
----------------------------------------

Article: Apple stock rises 5% after strong quarterly earnings report...
Expected: Business
Predicted: {"category":"Sci/Tech"}
----------------------------------------

Article: Google releases new AI model that can generate realistic ima...
Expected: Sci/Tech
Predicted: {"category":"Sci/Tech"}
----------------------------------------


## 10. Next Steps

Now that training is complete:

### 1. Full Evaluation
Run the fine-tuned model through the same evaluation as the base model:
- Use `base_model_performance.ipynb` as a template
- Load the fine-tuned model from `checkpoints/qwen7b-ag-news-full/final`
- Compare accuracy, F1, confusion matrix

### 2. Serve with vLLM
To serve the fine-tuned model:
```bash
# Update docker-compose-qwen7b.yml to point to fine-tuned model:
# command: vllm serve /checkpoints/qwen7b-ag-news-full/final ...

# Then start the server:
./start_docker.sh start qwen7b
```

### 3. Compare Results

| Metric | Base Model | Fine-Tuned | Improvement |
|--------|------------|------------|-------------|
| Accuracy | 78.63% | TBD | TBD |
| F1 (macro) | 77.80% | TBD | TBD |
| Sci/Tech Recall | 46.37% | TBD | TBD |

In [13]:
# Final summary
print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)
print(f"""
Model:              {MODEL_NAME}
Dataset:            {len(dataset):,} examples
Training time:      {training_time:.2f} hours
Output directory:   {OUTPUT_DIR}

Next steps:
1. Run full evaluation on test set
2. Compare with base model results
3. Serve with vLLM if results are good
""")

TRAINING SUMMARY

Model:              Qwen/Qwen2.5-7B-Instruct
Dataset:            120,000 examples
Training time:      9.72 hours
Output directory:   checkpoints/qwen7b-ag-news-full

Next steps:
1. Run full evaluation on test set
2. Compare with base model results
3. Serve with vLLM if results are good

